In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12973,2021-11-02T17:00:00,ITA,19,Sicilia,38.115697,13.362357,305,40,345,7266,...,DECEDUTI: N. 1 IL 01/11/21 - N. 1 IL 31/10/21 ...,6.0,Dalla data odierna per la rilevazione del dato...,NaN,308909.0,0.0,3365541.0,3368396.0,ITG,ITG1
12974,2021-11-02T17:00:00,ITA,9,Toscana,43.769231,11.255889,271,28,299,5603,...,NaN,1.0,NaN,NaN,285073.0,4958.0,4796760.0,2358256.0,ITI,ITI1
12975,2021-11-02T17:00:00,ITA,10,Umbria,43.106758,12.388247,41,7,48,1123,...,NaN,1.0,NaN,NaN,65266.0,0.0,1187851.0,1001481.0,ITI,ITI2
12976,2021-11-02T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,5,0,5,74,...,NaN,0.0,NaN,NaN,11450.0,865.0,112805.0,118749.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12973,2021-11-02T17:00:00,19,Sicilia,305,40,345,7266,7611,227,382,...,308909,6733937,2564359.0,6.0,308909.0,0.0,3365541.0,3368396.0,ITG,ITG1
12974,2021-11-02T17:00:00,9,Toscana,271,28,299,5603,5902,5,190,...,290031,7155016,3093777.0,1.0,285073.0,4958.0,4796760.0,2358256.0,ITI,ITI1
12975,2021-11-02T17:00:00,10,Umbria,41,7,48,1123,1171,11,20,...,65266,2189332,457101.0,1.0,65266.0,0.0,1187851.0,1001481.0,ITI,ITI2
12976,2021-11-02T17:00:00,2,Valle d'Aosta,5,0,5,74,79,-1,5,...,12315,231554,92749.0,0.0,11450.0,865.0,112805.0,118749.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-11-02', '2021-11-01')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-11-02T17:00:00,1,Piemonte,203,22,225,4090,4315,56,193,...,385248,7980163,2608388.0,1.0,361596.0,23652.0,3653414.0,4326749.0,ITC,ITC1
1,2021-11-02T17:00:00,2,Valle d'Aosta,5,0,5,74,79,-1,5,...,12315,231554,92749.0,0.0,11450.0,865.0,112805.0,118749.0,ITC,ITC2
2,2021-11-02T17:00:00,3,Lombardia,293,48,341,9517,9858,174,213,...,894982,17517039,5825284.0,4.0,827297.0,67685.0,11302165.0,6214874.0,ITC,ITC4
3,2021-11-02T17:00:00,5,Veneto,177,33,210,10793,11003,185,336,...,481158,13872127,2189658.0,2.0,465983.0,15175.0,7111155.0,6760972.0,ITH,ITH3
4,2021-11-02T17:00:00,6,Friuli Venezia Giulia,79,17,96,2162,2258,84,160,...,117460,3219101,860570.0,1.0,101836.0,15624.0,2248107.0,970994.0,ITH,ITH4
5,2021-11-02T17:00:00,7,Liguria,78,11,89,1156,1245,3,84,...,115065,2645520,916694.0,1.0,115065.0,0.0,1717446.0,928074.0,ITC,ITC3
6,2021-11-02T17:00:00,8,Emilia-Romagna,320,29,349,7262,7611,85,334,...,433011,9678618,2159800.0,2.0,432624.0,387.0,6177563.0,3501055.0,ITH,ITH5
7,2021-11-02T17:00:00,9,Toscana,271,28,299,5603,5902,5,190,...,290031,7155016,3093777.0,1.0,285073.0,4958.0,4796760.0,2358256.0,ITI,ITI1
8,2021-11-02T17:00:00,10,Umbria,41,7,48,1123,1171,11,20,...,65266,2189332,457101.0,1.0,65266.0,0.0,1187851.0,1001481.0,ITI,ITI2
9,2021-11-02T17:00:00,11,Marche,60,20,80,2175,2255,-1,42,...,116319,1633556,947135.0,2.0,116319.0,0.0,1404095.0,229461.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-11-01T17:00:00,1,Piemonte,188,21,209,4050,4259,9,88,...,385055,7935613,2605215.0,2.0,361504.0,23551.0,3650400.0,4285213.0,ITC,ITC1
1,2021-11-01T17:00:00,2,Valle d'Aosta,5,0,5,75,80,0,0,...,12310,230715,92610.0,0.0,11445.0,865.0,112699.0,118016.0,ITC,ITC2
2,2021-11-01T17:00:00,3,Lombardia,288,45,333,9351,9684,64,186,...,894769,17474420,5819595.0,3.0,827159.0,67610.0,11293748.0,6180672.0,ITC,ITC4
3,2021-11-01T17:00:00,5,Veneto,170,31,201,10617,10818,232,362,...,480822,13856391,2187735.0,2.0,465658.0,15164.0,7104177.0,6752214.0,ITH,ITH3
4,2021-11-01T17:00:00,6,Friuli Venezia Giulia,75,18,93,2081,2174,25,72,...,117301,3210975,860004.0,4.0,101693.0,15608.0,2246440.0,964535.0,ITH,ITH4
5,2021-11-01T17:00:00,7,Liguria,75,10,85,1157,1242,61,65,...,114981,2640061,915897.0,0.0,114981.0,0.0,1715998.0,924063.0,ITC,ITC3
6,2021-11-01T17:00:00,8,Emilia-Romagna,309,30,339,7187,7526,97,407,...,432679,9662656,2158416.0,2.0,432294.0,385.0,6171073.0,3491583.0,ITH,ITH5
7,2021-11-01T17:00:00,9,Toscana,255,28,283,5614,5897,101,204,...,289841,7140506,3089746.0,3.0,284901.0,4940.0,4791946.0,2348560.0,ITI,ITI1
8,2021-11-01T17:00:00,10,Umbria,38,6,44,1116,1160,-13,13,...,65246,2184136,456836.0,0.0,65246.0,0.0,1186971.0,997165.0,ITI,ITI2
9,2021-11-01T17:00:00,11,Marche,54,18,72,2184,2256,4,45,...,116277,1631529,945796.0,0.0,116277.0,0.0,1403074.0,228455.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-11-02T17:00:00,1,Piemonte,203,22,225,4090,4315,56,193,...,7980163,2608388.0,1.0,361596.0,23652.0,3653414.0,4326749.0,ITC,ITC1,4.341375e+06
1,2021-11-02T17:00:00,2,Valle d'Aosta,5,0,5,74,79,-1,5,...,231554,92749.0,0.0,11450.0,865.0,112805.0,118749.0,ITC,ITC2,1.255010e+05
2,2021-11-02T17:00:00,3,Lombardia,293,48,341,9517,9858,174,213,...,17517039,5825284.0,4.0,827297.0,67685.0,11302165.0,6214874.0,ITC,ITC4,1.010397e+07
3,2021-11-02T17:00:00,5,Veneto,177,33,210,10793,11003,185,336,...,13872127,2189658.0,2.0,465983.0,15175.0,7111155.0,6760972.0,ITH,ITH3,4.907704e+06
4,2021-11-02T17:00:00,6,Friuli Venezia Giulia,79,17,96,2162,2258,84,160,...,3219101,860570.0,1.0,101836.0,15624.0,2248107.0,970994.0,ITH,ITH4,1.211357e+06
5,2021-11-02T17:00:00,7,Liguria,78,11,89,1156,1245,3,84,...,2645520,916694.0,1.0,115065.0,0.0,1717446.0,928074.0,ITC,ITC3,1.543127e+06
6,2021-11-02T17:00:00,8,Emilia-Romagna,320,29,349,7262,7611,85,334,...,9678618,2159800.0,2.0,432624.0,387.0,6177563.0,3501055.0,ITH,ITH5,4.467118e+06
7,2021-11-02T17:00:00,9,Toscana,271,28,299,5603,5902,5,190,...,7155016,3093777.0,1.0,285073.0,4958.0,4796760.0,2358256.0,ITI,ITI1,3.722729e+06
8,2021-11-02T17:00:00,10,Umbria,41,7,48,1123,1171,11,20,...,2189332,457101.0,1.0,65266.0,0.0,1187851.0,1001481.0,ITI,ITI2,8.802850e+05
9,2021-11-02T17:00:00,11,Marche,60,20,80,2175,2255,-1,42,...,1633556,947135.0,2.0,116319.0,0.0,1404095.0,229461.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-11-02T17:00:00,1,Piemonte,203,22,225,4090,4315,56,193,...,4326749.0,ITC,ITC1,4.341375e+06,Piemonte,203,5824,22,628,99
1,2021-11-02T17:00:00,2,Valle d'Aosta,5,0,5,74,79,-1,5,...,118749.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,5,83,0,33,0
2,2021-11-02T17:00:00,3,Lombardia,293,48,341,9517,9858,174,213,...,6214874.0,ITC,ITC4,1.010397e+07,Lombardia,293,6369,48,1530,0
3,2021-11-02T17:00:00,5,Veneto,177,33,210,10793,11003,185,336,...,6760972.0,ITH,ITH3,4.907704e+06,Veneto,177,6000,33,1000,0
4,2021-11-02T17:00:00,6,Friuli Venezia Giulia,79,17,96,2162,2258,84,160,...,970994.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,79,1277,17,175,0
5,2021-11-02T17:00:00,7,Liguria,78,11,89,1156,1245,3,84,...,928074.0,ITC,ITC3,1.543127e+06,Liguria,78,1715,11,217,12
6,2021-11-02T17:00:00,8,Emilia-Romagna,320,29,349,7262,7611,85,334,...,3501055.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,320,7920,29,889,0
7,2021-11-02T17:00:00,9,Toscana,271,28,299,5603,5902,5,190,...,2358256.0,ITI,ITI1,3.722729e+06,Toscana,271,5033,28,570,36
8,2021-11-02T17:00:00,10,Umbria,41,7,48,1123,1171,11,20,...,1001481.0,ITI,ITI2,8.802850e+05,Umbria,41,662,7,84,43
9,2021-11-02T17:00:00,11,Marche,60,20,80,2175,2255,-1,42,...,229461.0,ITI,ITI3,1.518400e+06,Marche,60,964,20,212,61


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-11-02T17:00:00,12,Lazio,433,58,491,9236,9727,-23,388,...,6.15,-50,-57,21386,3406.0,5,406,1.81,0.00661,6.75
1,2021-11-02T17:00:00,19,Sicilia,305,40,345,7266,7611,227,382,...,4.48,218,87,13028,3711.0,3,152,2.93,0.00769,6.22
2,2021-11-02T17:00:00,5,Veneto,177,33,210,10793,11003,185,336,...,3.30,176,-26,15736,1923.0,1,150,2.14,0.00685,9.80
3,2021-11-02T17:00:00,8,Emilia-Romagna,320,29,349,7262,7611,85,334,...,3.26,75,-73,15962,1384.0,8,239,2.09,0.00748,9.69
4,2021-11-02T17:00:00,15,Campania,269,22,291,8844,9135,-15,248,...,3.96,-28,-106,11491,1423.0,10,253,2.16,0.00429,8.09
5,2021-11-02T17:00:00,3,Lombardia,293,48,341,9517,9858,174,213,...,3.14,166,27,42619,5689.0,0,39,0.50,0.00211,8.86
6,2021-11-02T17:00:00,1,Piemonte,203,22,225,4090,4315,56,193,...,3.50,40,105,44550,3173.0,1,136,0.43,0.00445,8.87
7,2021-11-02T17:00:00,9,Toscana,271,28,299,5603,5902,5,190,...,4.91,-11,-14,14510,4031.0,4,181,1.31,0.00510,7.79
8,2021-11-02T17:00:00,6,Friuli Venezia Giulia,79,17,96,2162,2258,84,160,...,9.71,81,88,8126,566.0,4,71,1.97,0.01321,9.70
9,2021-11-02T17:00:00,7,Liguria,78,11,89,1156,1245,3,84,...,5.07,-1,19,5459,797.0,2,79,1.54,0.00544,7.46


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Friuli Venezia Giulia,17,158,175,9.71
1,Marche,20,192,212,9.43
2,Umbria,7,77,84,8.33
3,Lazio,58,885,943,6.15
4,Molise,2,37,39,5.13
5,Liguria,11,206,217,5.07
6,Abruzzo,9,172,181,4.97
7,Toscana,28,542,570,4.91
8,Sicilia,40,852,892,4.48
9,Puglia,20,462,482,4.15


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))